In [1]:
import json
from pydantic import BaseModel, validator
from typing import Literal

# Define the Pydantic models for the SierraQueryBuilder

class Target(BaseModel):
    record_type: Literal['bib', 'item']   # TODO more / others
    field_tag: str                        # TODO Literals? Fixed Fields?


class LogicalOperator(BaseModel):
    operator: Literal['and', 'or', 'not']  # TODO is this it?

    def json_serializable(self):
        return str(self.operator)

    def __str__(self):
        """
        When the object is printed or converted to a string, return its JSON representation.
        """
        return str(self.operator)


class Expression(BaseModel):
    operation: Literal['has', 'equals']
    operands: list

    def json_serializable(self):
        return self.model_dump()


class CompoundExpression(BaseModel):
    expressions: list  # This will hold Expression and LogicalOperator instances

    def json_serializable(self):
        # Convert each Expression or LogicalOperator to JSON serializable form
        return [item.json_serializable() if isinstance(item, (Expression, LogicalOperator)) else item for item in self.expressions]

    def add_expression(self, expression: Expression):
        if not self.expressions or isinstance(self.expressions[-1], LogicalOperator):
            self.expressions.append(expression)
        else:
            raise ValueError("Cannot add an Expression after another Expression without a LogicalOperator.")

    def add_logical_operator(self, operator: LogicalOperator):
        if self.expressions and isinstance(self.expressions[-1], Expression):
            self.expressions.append(operator)
        else:
            raise ValueError("Cannot add a LogicalOperator before adding an Expression or after another LogicalOperator.")

    def __str__(self):
        return json.dumps(self.json_serializable(), indent=2)


class Query(BaseModel):
    target: Target
    expr: CompoundExpression  # Changed from Expression to CompoundExpression

    def json_serializable(self):
        return {
            "target": self.target.dict(),
            "expr": self.expr.json_serializable()
        }

    def __str__(self):
        return json.dumps(self.json_serializable(), indent=2)


class SierraQueryBuilder:
    """
    Builds queries for Create Lists and SierraRESTAPI endpoints 
    """
    def __init__(self):
        self.query = {
            "queries": []
        }

    # def add_simple_query(
    #     self,
    #     record_type: str,  # Target
    #     field_tag: str,    # Target
    #     op: str,           # Expression
    #     operands: list     # Expression
    # ):
    #     target = Target(record_type=record_type, field_tag=field_tag)
    #     expression = Expression(op=op, operands=operands)
        
    #     query = Query(target=target, expression=expression)

    #     # Check if the queries list is empty or the last item is a LogicalOperator
    #     if not self.query["queries"] or isinstance(self.query["queries"][-1], LogicalOperator):
    #         self.query["queries"].append(query)

    # def add_logical_operator(self, operator: str):
    #     # Ensure that the previous (last) element of queries is a Query
    #     if self.query["queries"] and isinstance(self.query["queries"][-1], Query):
    #         self.query["queries"].append(LogicalOperator(operator=operator))  # Corrected line
    #     else:
    #         raise ValueError("Cannot add a LogicalOperator before adding a Query or after another LogicalOperator.")
    def add_simple_query(
        self,
        record_type: str,  # Target
        field_tag: str,    # Target
        operation: str,           # Expression
        operands: list     # Expression
    ):
        target = Target(record_type=record_type, field_tag=field_tag)
        expression = Expression(operation=operation, operands=operands)
        
        # Create a CompoundExpression with the single Expression
        compound_expression = CompoundExpression(expressions=[expression])
        
        # Create a Query with the CompoundExpression
        query = Query(target=target, expr=compound_expression)

        # Check if the queries list is empty or the last item is a LogicalOperator
        if not self.query["queries"] or isinstance(self.query["queries"][-1], LogicalOperator):
            self.query["queries"].append(query)
    def __str__(self):
        """
        Returns the string representation of the JSON query.
        :return: String representation of the JSON query.
        """
        # Convert all Query and LogicalOperator objects to their dictionary representations
        queries_list = [item.json_serializable() if isinstance(item, (Query, LogicalOperator)) else item for item in self.query["queries"]]
        
        # Serialize the list of dictionaries to a JSON string
        return json.dumps({"queries": queries_list}, indent=2)


In [2]:

q = SierraQueryBuilder()
# q.add_simple_query(record_type='bib', field_tag='a', operation='has', operands=['Hemingway'] )
# q.add_logical_operator('and')
# q.add_simple_query("bib", "t", "has", ["Old Man and the Sea"])
# print(q)

In [6]:
q.add_simple_query(record_type='bib', field_tag='a', operation='has', operands=['Hemingway'])

ValidationError: 1 validation error for Expression
operation
  field required (type=value_error.missing)

In [5]:
query_builder = SierraQueryBuilder()
query_builder.start_query('bib', 'a') \
    .add_expression('has', ['Hemingway']) \
    .end_query() \
    .add_operator('and') \
    .start_query('bib', 't') \
    .add_expression('has', ['Old Man and the Sea']) \
    .end_query()

print(query_builder)

AttributeError: 'SierraQueryBuilder' object has no attribute 'start_query'